In [11]:
import pandas as pd

melb_data_ps=pd.read_csv('data/melb_data_ps.csv', sep=',')

# создаём пустой список *ПОДРОБНОЕ ОПИСАНИЕ КОДА В ТЕТРЕДИ
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_data_ps.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_data_ps[col].nunique(), melb_data_ps[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts) # в результате видим что с определенного момента (строчка 9) видим резкий скачок роста уникальных значений(в теоритическом курсе их 152, хз где я чет натворил, но это не особо важно)

display(melb_data_ps.info()) # обращаем внимание на то что объем памати составляет 2.4 Мб

# Сделаем преобразование столбцов к типу данных category: *ПОДРОБНОЕ ОПИСАНИЕ КОДА В ТЕТРАДИ

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_data_ps.columns: # цикл по именам столбцов
    if melb_data_ps[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_data_ps[col] = melb_data_ps[col].astype('category') # преобразуем тип столбца
display(melb_data_ps.info()) # обращаем внимание на то что теперь объем памати составляет 1.9 Мб !!!!

# ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY
# специальный аксесcор cat
print(melb_data_ps['Regionname'].cat.categories)

# А теперь посмотрим, каким образом столбец кодируется в виде чисел в памяти компьютера. Для этого можно воспользоваться атрибутом codes:
display(melb_data_ps['Regionname'].cat.codes)

# С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. 
# Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.
# Рассмотрим на примере: переименуем категории признака типа постройки Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).
melb_data_ps['Type'] = melb_data_ps['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_data_ps['Type'])

# ПОДВОДНЫЕ КАМНИ
# Представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры (flat). 
# Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. 
# Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data, и выведем результат на экран:

new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_data_ps['Type'].dtype)
display(new_houses_types) # С нашими новыми объектами недвижимости произошло нечто странное. По какой-то причине вместо квартир мы получили пустые значения — NaN !!!

# !!!!!!!!!!!!!!!!!!!!!!!!!!
# На самом деле причина проста: тип данных category хранит только категории, которые были объявлены при его инициализации. 
# При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.
# !!!!!!!!!!!!!!!!!!!!!!!!!!

# Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

melb_data_ps['Type'] = melb_data_ps['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_data_ps['Type'].dtype)
display(new_houses_types)

# Добавление новой категории в столбец Type не отразится на самом столбце — текущие категории не изменятся, 
# однако такое преобразование позволит добавлять в таблицу новые данные о домах с новой категорией — flat.

#!!!!!!!!!!!!!!!!!!!!!!!
# Из данного примера можно сделать вывод, что если набор категорий в столбце жёстко не зафиксирован и может обновляться в процессе работы, 
# то тип category не является подходящим типом данных для этого столбца или необходимо постоянно писать проверки при обновлении таблицы.
#!!!!!!!!!!!!!!!!!!!!!!!

# PRACTICE:
display(melb_data_ps.info())
list_unic_val = melb_data_ps['Suburb'].value_counts().nlargest(118) # обозначил переменнную в которую вывел первые 119 уникальных элементов
melb_data_ps['Suburb']=melb_data_ps['Suburb'].apply(lambda x: x if x in list_unic_val else 'other') # прогнал весь столбец по условию уникальных значений 119
melb_data_ps['Suburb']=melb_data_ps['Suburb'].astype('category') # создал список который будет преобразован в cat
melb_data_ps.info()
display(melb_data_ps.info())

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,Bedroom,12,int64
7,CouncilArea,33,object
8,Date,58,object
9,YearBuilt,144,int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Address        13580 non-null  object 
 3   Rooms          13580 non-null  int64  
 4   Type           13580 non-null  object 
 5   Price          13580 non-null  float64
 6   Method         13580 non-null  object 
 7   SellerG        13580 non-null  object 
 8   Date           13580 non-null  object 
 9   Distance       13580 non-null  float64
 10  Postcode       13580 non-null  int64  
 11  Bedroom        13580 non-null  int64  
 12  Bathroom       13580 non-null  int64  
 13  Car            13580 non-null  int64  
 14  Landsize       13580 non-null  float64
 15  BuildingArea   13580 non-null  float64
 16  YearBuilt      13580 non-null  int64  
 17  CouncilArea    12211 non-null  object 
 18  Lattit

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None

...

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  category
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  object  
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  YearBuilt      13580 non-null  category
 17  CouncilArea    12211 non-null  

None